In [ ]:
!pip install datasets
!pip install tiktoken
!pip install gutenbergpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 14.3 MB/s eta 0:00:00
  Created wheel for httpsproxy-urllib2: filename=httpsproxy_urllib2-1.0-py3-none-any.whl size=29250 sha256=0ae0fe2bf147d0a30f8acf4a87f66c9cfdcbe07727233f8e68d988e61232d4c1
  Stored in directory: /root/.cache/pip/wheels/1b/fa/c3/4c14e72101070c40b56c2bfb4617e510e68f121e4f736a5d2a
Successfully built httpsproxy-urllib2


# Importing Necessary Libraries

In [ ]:
import re
import importlib
import tiktoken
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
import requests

# Getting Our Corpus

The notebook retrieves the full text of Moby-Dick from Project Gutenberg. The text is stored in the variable **corpus** for tokenization.

In [ ]:
url = "https://www.gutenberg.org/files/2701/2701-0.txt" # The  Moby Dick book will be used for this
response = requests.get(url)

corpus = response.text

# Creating Basic Tokenizer from Corpus

This initial version of the tokenizer processes the corpus by:
* Splitting the text using regular expressions to capture common punctuation and whitespace as tokens.
* Constructing a vocabulary as a sorted, unique list of words and punctuation marks from the corpus.
* Mapping each word to a unique ID in **vocab_to_id** for encoding and in **id_to_vocab** for decoding.

The tokenizer provides two main functions:
1. **encode(text)**: Converts a given text into a list of token IDs based on the vocabulary.
2. **decode(tokens)**: Reconstructs the original text from token IDs, reattaching punctuation correctly.

In [ ]:
class LLMBasicTokenizerV1:
  def __init__(self, corpus):
    processed_corpus = re.split(r'([,.;:?!_"()\']--|\s)', corpus)
    processed_corpus = [token.strip() for token in processed_corpus if token.strip()]

    vocab = sorted(set(processed_corpus))
    self.vocab_to_id = {word:i for i, word in enumerate(vocab)}
    self.id_to_vocab = {i:word for i, word in enumerate(vocab)}

  def encode(self, text):
    split_text = re.split(r'([,.;:?!_"()\']--|\s)', text)
    return [self.vocab_to_id[token.strip()] for token in split_text if token.strip() in self.vocab_to_id]

  def decode(self, tokens):
    joined_tokens = " ".join([self.id_to_vocab[token] for token in tokens])
    return re.sub(r'\s+([,.;:?!_"()\'])', r'\1', joined_tokens)


In [ ]:
tokenizer = LLMBasicTokenizerV1(corpus)
text = """Can you please, please give me your name so I can see who you actually are."""
token_ids = tokenizer.encode(text)
decoded_text = tokenizer.decode(token_ids)
print(token_ids)
print(decoded_text)

[1046, 32687, 22807, 22805, 14878, 19774, 32705, 20703, 26968, 2365, 8371, 25732, 32086, 32687, 5284, 6145]
Can you please, please give me your name so I can see who you actually are.


# Creating a More Advanced Tokenizer

* This version extends LLMBasicTokenizerV1 by introducing special tokens, <|endoftext|> and <|unk|>, to handle text boundaries and unknown words.
* This tokenizer allows better handling of unknown or out-of-vocabulary tokens by introducing an unknown token placeholder.
* Like LLMBasicTokenizerV1, it provides the encode and decode methods, enhancing flexibility for more robust tokenization.

In [ ]:
class LLMBasicTokenizerV2:
  def __init__(self, corpus):
    processed_corpus = re.split(r'([,.;:?!_"()\']--|\s)', corpus)
    processed_corpus = [token.strip() for token in processed_corpus if token.strip()]

    vocab = sorted(set(processed_corpus))
    vocab.extend(["<|endoftext|>", "<|unk|>"])
    self.vocab_to_id = {word:i for i, word in enumerate(vocab)}
    self.id_to_vocab = {i:word for i, word in enumerate(vocab)}

  def encode(self, text):
    split_text = re.split(r'([,.;:?!_"()\']--|\s)', text)
    return [self.vocab_to_id[token.strip()] for token in split_text if token.strip() in self.vocab_to_id]

  def decode(self, tokens):
    joined_tokens = " ".join([self.id_to_vocab[token] for token in tokens])
    return re.sub(r'\s+([,.;:?!_"()\'])', r'\1', joined_tokens)


In [ ]:
tokenizer = LLMBasicTokenizerV2(corpus)
text = """Timothy, can you please, please give me your name so I can see who you actually are."""
token_ids = tokenizer.encode(text)
decoded_text = tokenizer.decode(token_ids)
print(token_ids)
print(decoded_text)

[8371, 32687, 22807, 22805, 14878, 19774, 32705, 20703, 26968, 2365, 8371, 25732, 32086, 32687, 5284, 6145]
can you please, please give me your name so I can see who you actually are.
